Задача:

Производственная система состоит из m станков,каждый из которых подвержен случайным поломкам.  Перед поломкой станок работает в течение промежутка времени, который представляет собой экспоненциально распределённую случайную величину со средним значением 8 часов.  Ремонтом станков занимается s ремонтников(где s — фиксированное положительное целое число). Для того, чтобы починить один станок, одному ремонтнику требуется экспоненциально распределенный промежуток времени со средним значением 2 часа; Ремонтом сломанного станка может заниматься только один ремонтник, даже если в это время есть другие свободные ремонтники. Если одновременно ломается более s станков, они формируют очередь на ремонт с дисциплиной FIFO и ждут первого свободного ремонтника.  Далее ремонтник чинит сломанный станок до тех пор, пока он не будет починен, независимо от того, что еще происходит в системе.  Предположим, что каждый час поломки одного станка стоит 50 долларов, а час работы ремонтника стоит 10 долларов.  (Ремонтникам платят почасовую оплату независимо от того, работают ли они на самом деле.) 

Предположим, что количество станков в системе равно m = 5, но напишите общий код, позволяющий учитывать значение m, достигающее 20, путем изменения входного параметра.

Смоделируйте систему ровно при 800 часах работы для каждой стратегии занятости ( s = 1, 2, 3, 4, 5 ) ,  чтобы определить, какая из них приводит к наименьшим ожидаемым средним затратам в час, допустив, что в момент времени 0 все машины только что были отремонтированы.

In [1]:
import numpy as np

def exponential_distribution(mean, size):
    return np.random.exponential(mean, size)

def simulate_system(m, s, hours):
    machine_working_time = exponential_distribution(8, m)
    machine_breakdown_time = np.zeros(m)
    total_cost = 0

    for hour in range(hours):
        for i in range(m):
            if machine_breakdown_time[i] == 0:
                machine_working_time[i] -= 1
                if machine_working_time[i] <= 0:
                    machine_breakdown_time[i] = exponential_distribution(8, 1)
            else:
                machine_breakdown_time[i] -= 1
                if machine_breakdown_time[i] <= 0:
                    machine_breakdown_time[i] = 0
                    machine_working_time[i] = exponential_distribution(8, 1)
                    total_cost += 50

        repair_queue = 0
        for i in range(m):
            if machine_breakdown_time[i] > 0:
                repair_queue += 1

        while repair_queue > 0:
            repair_time = exponential_distribution(2, 1)
            total_cost += s * 10
            repair_queue -= s
            if repair_queue < 0:
                total_cost -= (abs(repair_queue) * 10)

    return total_cost / hours


m_values = range(1, 21, 1)
s_values = range (1, 6, 1)

for m in m_values:
    for s in s_values:
        avg_cost = simulate_system(m, s, 800)
        print(f"Для кол-ва станков m = {m} и стратегии занятости s = {s}, ожидаемая средняя затрата в час: {avg_cost}")

/tmp/ipykernel_70078/2524880457.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  machine_breakdown_time[i] = exponential_distribution(8, 1)
/tmp/ipykernel_70078/2524880457.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  machine_working_time[i] = exponential_distribution(8, 1)


Для кол-ва станков m = 1 и стратегии занятости s = 1, ожидаемая средняя затрата в час: 7.8875
Для кол-ва станков m = 1 и стратегии занятости s = 2, ожидаемая средняя затрата в час: 7.7375
Для кол-ва станков m = 1 и стратегии занятости s = 3, ожидаемая средняя затрата в час: 8.4375
Для кол-ва станков m = 1 и стратегии занятости s = 4, ожидаемая средняя затрата в час: 8.2
Для кол-ва станков m = 1 и стратегии занятости s = 5, ожидаемая средняя затрата в час: 7.5875
Для кол-ва станков m = 2 и стратегии занятости s = 1, ожидаемая средняя затрата в час: 15.6
Для кол-ва станков m = 2 и стратегии занятости s = 2, ожидаемая средняя затрата в час: 15.8
Для кол-ва станков m = 2 и стратегии занятости s = 3, ожидаемая средняя затрата в час: 15.5
Для кол-ва станков m = 2 и стратегии занятости s = 4, ожидаемая средняя затрата в час: 15.7125
Для кол-ва станков m = 2 и стратегии занятости s = 5, ожидаемая средняя затрата в час: 15.2375
Для кол-ва станков m = 3 и стратегии занятости s = 1, ожидаемая сре

In [38]:
for m in m_values:
    for s in s_values:
        avg_cost = simulate_system(m, s, 800)
        print( m, s, avg_cost)

/tmp/ipykernel_42271/3750290773.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  machine_breakdown_time[i] = exponential_distribution(8, 1)
/tmp/ipykernel_42271/3750290773.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  machine_working_time[i] = exponential_distribution(8, 1)


1 1 7.775
1 2 8.1875
1 3 8.0375
1 4 7.6125
1 5 7.725
2 1 14.8375
2 2 15.875
2 3 16.425
2 4 15.3
2 5 15.675
3 1 23.3625
3 2 22.7
3 3 24.1125
3 4 23.375
3 5 23.85
4 1 33.225
4 2 32.6875
4 3 32.5625
4 4 31.5625
4 5 31.45
5 1 38.6375
5 2 39.5875
5 3 40.725
5 4 38.7875
5 5 38.2875
6 1 47.7625
6 2 47.375
6 3 47.55
6 4 46.2125
6 5 47.4375
7 1 57.0
7 2 54.0875
7 3 53.825
7 4 56.4125
7 5 55.3875
8 1 64.125
8 2 65.6
8 3 65.575
8 4 67.85
8 5 65.0
9 1 72.5
9 2 72.025
9 3 73.125
9 4 70.5375
9 5 71.6
10 1 77.6875
10 2 79.85
10 3 81.5375
10 4 79.6125
10 5 79.4375
11 1 86.975
11 2 87.95
11 3 87.3
11 4 88.275
11 5 88.6375
12 1 98.4875
12 2 95.6625
12 3 94.7375
12 4 96.975
12 5 91.6125
13 1 102.2
13 2 100.8
13 3 100.775
13 4 103.7625
13 5 103.975
14 1 112.325
14 2 104.8875
14 3 110.05
14 4 108.525
14 5 109.0
15 1 118.675
15 2 121.9625
15 3 118.125
15 4 117.875
15 5 117.35
16 1 124.1625
16 2 125.325
16 3 123.4125
16 4 126.0625
16 5 126.7625
17 1 136.375
17 2 133.45
17 3 133.025
17 4 135.525
17 5 137.4125

In [4]:
import pandas as pd
data = pd.read_csv('output_1.txt', sep = ' ', header = None)
data.columns = ['m', 's', 'avg_cost']
data

,m,s,avg_cost
0,1,1,7.7750
1,1,2,8.1875
2,1,3,8.0375
3,1,4,7.6125
4,1,5,7.7250
...,...,...,...
95,20,1,161.2375
96,20,2,158.6125
97,20,3,159.2625
98,20,4,155.5750
